In [93]:
import pycountry
import pandas as pd
import numpy as np
import re
import os

In [94]:
df = pd.read_csv('/Users/serenay/Documents/work/engagement_lead/COSMOS-request-dashboard/research/notebooks/data/COSMOS_Requests.csv')

In [95]:
# Null rows are deleted from 'Email' column
df = df[~df["Email"].isnull()]

In [96]:
# regex patterns
df_country_patterns = pd.read_csv("/Users/serenay/Documents/work/engagement_lead/COSMOS-request-dashboard/research/notebooks/data/country_regex_patterns.csv")

In [97]:
def match_country_pattern(email, df_country_patterns):
    '''`function returns a country name when it is matched with the regex pattern. '''
    for index, row in df_country_patterns.iterrows():
        if re.search(row['regex_pattern'], email):
            return(row['country_name'])
            break

In [98]:
# Create 'country_name' for countries, we found from emails.
df["country_name"] = df["Email"].apply(lambda x: match_country_pattern(x, df_country_patterns))

In [99]:
# Create a csv file for null rows country_names column has.
# This file was created to figure out how many countries was written in the spreadsheet 
# before we determined the regex pattern from emails.
df[df["country_name"].isnull()].to_csv("/Users/serenay/Documents/work/engagement_lead/COSMOS-request-dashboard/research/notebooks/data/temp.csv")

In [100]:
# Drop duplicated Emails
df_missing_countries = pd.read_csv("/Users/serenay/Documents/work/engagement_lead/COSMOS-request-dashboard/research/notebooks/data/temp.csv").drop_duplicates(subset= "Email",
                                                             ignore_index=True)

In [101]:
# Select only two columns
df_missing_countries = df_missing_countries[["Email", "Country"]]


In [102]:
# Joined two df on email key column.
temp_joined_df = df.merge(df_missing_countries, on='Email', how='left')

In [103]:
# Combine two columns (country_name and country_y) into one (country_name_new)
temp_joined_df['country_name_new'] = temp_joined_df["country_name"].combine_first(temp_joined_df["Country_y"])

In [104]:
# Dropped unnecessary columns from dataframe
dropped_temp_joined_df=temp_joined_df.drop(columns=['Counry', 'Country_x', 'Domain'])

In [105]:
def convert_country_iso3(country):
    """converts country name to country codes"""
    try:
        iso3 = pycountry.countries.get(name=country).alpha_3
    except AttributeError:
        iso3 = "Other"
    return iso3

In [106]:
dropped_temp_joined_df.loc[dropped_temp_joined_df["country_name_new"].isnull(),'country_name_new'] = 'NaN'

In [107]:
# Add iso3 column for graph.
dropped_temp_joined_df["country_iso3"] = dropped_temp_joined_df["country_name_new"].apply(lambda x:convert_country_iso3(x))

In [108]:
geo_df = dropped_temp_joined_df['country_name_new'].value_counts().reset_index()

In [109]:
geo_df["country_iso3"] = geo_df["index"].apply(lambda x:convert_country_iso3(x))

In [110]:
geo_df

,index,country_name_new,country_iso3
0,United Kingdom,1322,GBR
1,NaN,584,Other
2,United States,344,USA
3,Canada,82,CAN
4,Australia,78,AUS
...,...,...,...
63,Algeria,1,DZA
64,Viet Nam,1,VNM
65,Morocco,1,MAR
66,Costa Rica,1,CRI


In [111]:
geo_df.columns= ["country", "download_count", "country_iso3"]

In [112]:
geo_df

,country,download_count,country_iso3
0,United Kingdom,1322,GBR
1,NaN,584,Other
2,United States,344,USA
3,Canada,82,CAN
4,Australia,78,AUS
...,...,...,...
63,Algeria,1,DZA
64,Viet Nam,1,VNM
65,Morocco,1,MAR
66,Costa Rica,1,CRI


In [113]:
geo_df.to_csv("/Users/serenay/Documents/work/engagement_lead/COSMOS-request-dashboard/interactive-geo-map/data/cosmos_download.csv")